In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

from langchain_community.vectorstores import SupabaseVectorStore
from supabase.client import Client, create_client

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

E0000 00:00:1759866925.108644 4975465 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1759866925.112832 4975465 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [4]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

loader = PyPDFLoader("../../data/TPC-H_v3.0.1.pdf")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [7]:
len(docs)

138

In [ ]:
# import time

# rate_limit = 10 # (RPM - 100; TPM - 30,000; RPD - 1000)
# rate_limit_period = 120
# for i in range(0, len(docs), rate_limit):
#     print(f"embedding documents {i}-{i+rate_limit-1}")
#     docs_chunk = docs[i:i+rate_limit]
#     vector_store = SupabaseVectorStore.from_documents(
#         docs_chunk,
#         embeddings,
#         client=supabase,
#         table_name="documents_temp",
#         query_name="match_documents_temp",
#         chunk_size=500
#     )
#     time.sleep(rate_limit_period)

embedding documents 50-59
embedding documents 60-69
embedding documents 70-79
embedding documents 80-89
embedding documents 90-99
embedding documents 100-109
embedding documents 110-119
embedding documents 120-129
embedding documents 130-139


In [12]:
vector_store = SupabaseVectorStore(
    embedding=embeddings,
    client=supabase,
    table_name="documents_temp",
    query_name="match_documents_temp",
)

In [13]:
query = "what is the business and application environment of the TPCH-H database?"
matched_docs = vector_store.similarity_search(query)

In [14]:
print(matched_docs[0].page_content)

TPC BenchmarkTM H Standard Specification Revision 3.0.1 Page 11 
1:  LOGICAL DATABASE DESIGN 
1.1 Business and Application Environment 
TPC Benchmark™ H is comprised of a set of business queries designed to exercise system functionalities in a 
manner representative of complex business analysis applications. T hese queries have been given a realistic context, 
portraying the activity of a wholesale supplier to help the reader relate intuitively to the components of the 
benchmark. 
 
TPC-H does not represent the activity of any particular business segment, but rathe r any industry which must 
manage sell, or distribute a product worldwide (e.g., car rental, food distribution, parts, suppliers, etc.). TPC -H does 
not attempt to be a model of how to build an actual information analysis application.  
 
The purpose of this be nchmark is to reduce the diversity of operations found in an information analysis application, 
while retaining the application's essential performance characterist

In [15]:
matched_docs = vector_store.similarity_search_with_relevance_scores(query)

In [16]:
matched_docs[0]

(Document(metadata={'page': 10, 'title': '0: INTRODUCTION', 'author': 'John Galloway', 'source': '../../data/TPC-H_v3.0.1.pdf', 'creator': 'Microsoft® Word for Microsoft 365', 'moddate': '2022-04-28T16:01:40-07:00', 'producer': 'Microsoft® Word for Microsoft 365', 'page_label': '11', 'total_pages': 138, 'creationdate': '2022-04-28T16:01:40-07:00'}, page_content="TPC BenchmarkTM H Standard Specification Revision 3.0.1 Page 11 \n1:  LOGICAL DATABASE DESIGN \n1.1 Business and Application Environment \nTPC Benchmark™ H is comprised of a set of business queries designed to exercise system functionalities in a \nmanner representative of complex business analysis applications. T hese queries have been given a realistic context, \nportraying the activity of a wholesale supplier to help the reader relate intuitively to the components of the \nbenchmark. \n \nTPC-H does not represent the activity of any particular business segment, but rathe r any industry which must \nmanage sell, or distribute

In [17]:
retriever = vector_store.as_retriever()

In [18]:
matched_docs = retriever.invoke(query)

I0000 00:00:1759868984.353339 4975465 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


In [19]:
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)


## Document 0

TPC BenchmarkTM H Standard Specification Revision 3.0.1 Page 11 
1:  LOGICAL DATABASE DESIGN 
1.1 Business and Application Environment 
TPC Benchmark™ H is comprised of a set of business queries designed to exercise system functionalities in a 
manner representative of complex business analysis applications. T hese queries have been given a realistic context, 
portraying the activity of a wholesale supplier to help the reader relate intuitively to the components of the 
benchmark. 
 
TPC-H does not represent the activity of any particular business segment, but rathe r any industry which must 
manage sell, or distribute a product worldwide (e.g., car rental, food distribution, parts, suppliers, etc.). TPC -H does 
not attempt to be a model of how to build an actual information analysis application.  
 
The purpose of this be nchmark is to reduce the diversity of operations found in an information analysis application, 
while retaining the application's essential performa